## Digit Inference based on QML Model

In [ ]:
from skimage.transform import resize
import pickle
import numpy as np
from sklearn.decomposition import TruncatedSVD
import umap

In [13]:
def call_bacak_vqc(eval_count, var_params, eval_val, index):
    print("eval_count: {}".format(eval_count))
    print("var_params: {}".format(var_params))
    print("evl_val: {}".format(eval_val))
    print("index: {}".format(index))

In [14]:
models_dir = './models'
model_0_3_5_6_8 = pickle.load(open(f'{models_dir}/model_0_3_5_6_8','rb'))
model_0 = pickle.load(open(f'{models_dir}/model_0','rb'))
model_4 = pickle.load(open(f'{models_dir}/model_4','rb'))
model_5 = pickle.load(open(f'{models_dir}/model_5','rb'))
model_3_8 = pickle.load(open(f'{models_dir}/model_3_8','rb'))
model_3 = pickle.load(open(f'{models_dir}/model_3','rb'))
model_1_2 = pickle.load(open(f'{models_dir}/model_1_2','rb'))
model_1 = pickle.load(open(f'{models_dir}/model_1','rb'))
model_4_9 = pickle.load(open(f'{models_dir}/model_4_9','rb'))
# tsvd = pickle.load(open(f'{models_dir}/tsvd_transforms.pkl','rb'))
# um = pickle.load(open(f'{models_dir}/umap_transforms.pkl','rb'))
data_path = "./"
train_data = np.loadtxt(data_path + "mnist_train.csv", delimiter=",")
train_data_features = train_data[:10000, 1:]
tsvd = TruncatedSVD(n_components=20)
X_SVD = tsvd.fit_transform(train_data_features)
um = umap.UMAP()
_ = um.fit_transform(X_SVD)

In [7]:
model_0_3_5_6_8.predict()

In [69]:
from qiskit import *

def run_model(model,data):
    backend = BasicAer.get_backend(backend_value)
    if(backend_value=='IBMQ'):
        IBMQ.load_account()
        provider = IBMQ.get_provider(hub='ibm-q')
        backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= (n+1) and
                                       not x.configuration().simulator and x.status().operational==True))
#         print("least busy backend: ", backend)
    model.backend = backend
    model.predict(data)
    
    
    

def remove_padding(img):
    top_border = 0
    bot_border = img.shape[0]-1
    left_border = 0
    right_border = img.shape[1]-1
    for i in range(img.shape[0]):
        if(sum(img[i])!=0):
            top_border = i
            break
    for i in range(img.shape[0]-1,-1,-1):
        if(sum(img[i])!=0):
            bot_border = i
            break
            
    for i in range(img.shape[1]):
        if(sum(img[:,i])!=0):
            left_border = i
            break
    for i in range(img.shape[1]-1,-1,-1):
        if(sum(img[:,i])!=0):
            right_border = i
            break
    return img[top_border:bot_border,left_border:right_border]
    
    
def pre_process(img):
    img = remove_padding(img)
    img = resize(img, (24, 24))
    img_bl = np.zeros((28,28))
    img_bl[2:26,2:26] = img
    img = img_bl
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            img[i][j] = img[i][j] if img[i][j]<1 else 255

    flat = img.reshape(784,)

    # Apply Dimensionality Reduction

    X1 = tsvd.transform([flat])
    X2 = um.transform(X1)
    
    return X2
    
def predict(img):
    img = pre_process(img)
    pred = run_model(model_0_3_5_6_8,img)
    if(pred==1):
        pred = run_model(model_1_2,img)
        if(pred==1):
            pred = run_model(model_4_9,img)
            if(pred==1):
                return 7
            else:
                pred = run_model(model_4,img)
                return 4 if pred==0 else 9
        else:
            pred = run_model(model_1,img)
            return 1 if pred==0 else 2
    else:
        pred = run_model(model_3_8,img)
        if(pred==1):
            pred = run_model(model_5,img)
            if(pred==1):
                pred = run_model(model_0,img)
                return 0 if pred==0 else 6
            else:
                return 5
        else:
            pred = run_model(model_3,img)
            return 3 if pred==0 else 8
            
        

In [70]:
import jupyter_drawing_pad as jd
import numpy as np
import ipywidgets as widgets

radio_widget = widgets.RadioButtons(
    options=['qasm_simulator', 'statevector_simulator', 'IBMQ'],
    description='Choose Backend',
    disabled=False
)
radio_widget

RadioButtons(description='Choose Backend', options=('qasm_simulator', 'statevector_simulator', 'IBMQ'), value=…

In [71]:
widget = jd.CustomBox()
canvas_dim = (500,250)
image_dim = (250,250)
backend_value = radio_widget.value
widget



CustomBox(children=(DrawingPad(data=[[], [], []]), VBox(children=(Text(value='', description='Name:', placehol…

In [72]:
def get_image(canvas_data):
    image = np.zeros(image_dim)
    n = len(canvas_data[0])
    max_intensity = max(canvas_data[2])
    
    for i in range(n):
        x,y = int(canvas_data[0][i]/canvas_dim[0] * image_dim[0]), int(canvas_data[1][i]/canvas_dim[1] * image_dim[1])
        value = float(canvas_data[2][i])/float(max_intensity) * 255.0
        image[x][y] = value
#         print(f"{x},{y} : {value}")
    return image

In [74]:
num = predict(get_image(widget.drawing_pad.data))
print('You typed:',num)

You typed: 8
